In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import reconnect_degree_2
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes
from hyphae_id import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends, solve_degree4, clean_obvious_fake_tips
from extract_graph import prune_graph



SyntaxError: invalid syntax (hyphae_id.py, line 7)

In [ ]:
plate = 130
exp_clean = pickle.load(open( f'Data/'+f"experiment_{plate}.pick", "rb" ) )

In [ ]:
solve_two_ends = resolve_ambiguity_two_ends(exp_clean.hyphaes)
solved = solve_degree4(exp_clean)

In [ ]:
clean_obvious_fake_tips(exp_clean)

In [146]:
disapearing_hyph = [hyph for hyph in exp_clean.hyphaes if len(hyph.ts)>=1 and hyph.ts[-1]!=22 and hyph not in hyph_tiptip_set]

In [147]:
disapearing_hyph_len1 = [hyph for hyph in exp_clean.hyphaes if len(hyph.end.ts())==1 and hyph.ts[-1]!=22 and hyph not in hyph_tiptip_set]

In [148]:
len(disapearing_hyph),len(disapearing_hyph_len1)

(433, 0)

In [159]:
hypha = choice (disapearing_hyph)
plt.close('all')
hypha.ts,hypha

([0, 1, 2], Hyphae(775,778))

In [160]:
exp_clean.plot([1,2,3],[[hypha.root.label,hypha.end.label]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
hyph_tiptip_set = {c[0] for c in hyph_anas_tip_tip}

In [10]:
for hyph in exp_clean.hyphaes:
    hyph.update_ts()

In [4]:
number_face=[2+len(g.edges)-len(g.nodes) for g in exp_clean.nx_graph]

In [48]:
number_anas_theory = [number_face[i+1]-number_face[i] for i in range(len(number_face)-2)]

In [49]:
number_face

[430,
 454,
 488,
 474,
 499,
 535,
 555,
 588,
 591,
 624,
 661,
 667,
 689,
 704,
 729,
 744,
 782,
 814,
 850,
 850,
 855,
 869,
 941]

In [15]:
hyph_anas_tip_hyph = [hyphat for hyphat in exp_clean.hyphaes if len(hyphat.ts)>=1 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]
hyph_anas_tip_tip = []
potential = []
for hyph in exp_clean.hyphaes:
    if len(hyph.ts)>=2 and hyph.end.degree(hyph.ts[-1])==1 and hyph.end.ts()[-1]!=len(exp_clean.nx_graph)-1 and not np.all([hyph.get_length_pixel(t)<=20 for t in hyph.ts]):
        potential.append(hyph)
for hyph in potential:
    t0 = hyph.ts[-1]
    for hyph2 in potential:
        if hyph2.ts[-1] == t0 and hyph!=hyph2:
            vector = (hyph2.end.pos(t0)-hyph.end.pos(t0))/np.linalg.norm(hyph2.end.pos(t0)-hyph.end.pos(t0))
            vertical_vector=np.array([-1,0])
            dot_product = np.dot(vertical_vector,vector)
            if vertical_vector[1]*vector[0]-vertical_vector[0]*vector[1]>=0: #determinant
                angle = np.arccos(dot_product)/(2*np.pi)*360
            else:
                angle = -np.arccos(dot_product)/(2*np.pi)*360
            score = np.cos((angle-(180+hyph.end.edges(t0)[0].orientation_begin(t0,30)))/360*2*np.pi)+np.cos((360+angle-hyph2.end.edges(t0)[0].orientation_begin(t0,30))/360*2*np.pi)
            if np.linalg.norm(hyph2.end.pos(t0)-hyph.end.pos(t0))<=500 and score>=0.5:
                hyph_anas_tip_tip.append((hyph,hyph2,t0))

In [53]:
len(hyph_anas_tip_hyph)

239

In [60]:
hyph_anas_tip_tip[0][2]

8

In [52]:
np.sum(number_anas_theory)

439

In [43]:
def get_time_anas_tip_hyph(anas_tip_hyph):
    for t in anas_tip_hyph.ts:
        if anas_tip_hyph.end.degree(t)>=3:
            return(t-1)
def get_time_anas_tip_tip(anas_tip_tip):
    return(anas_tip_tip.ts[-1])

In [61]:
anas_tip_hyph_observed = np.array([len([hyph for hyph in hyph_anas_tip_hyph if get_time_anas_tip_hyph(hyph)==t]) for t in range(len(exp_clean.nx_graph)-2)])

In [93]:
anas_tip_tip_observed = np.array([len([hyphs for hyphs in hyph_anas_tip_tip if hyphs[2]==t])//2 for t in range(len(exp_clean.nx_graph)-2)])

In [94]:
anas_tip_hyph_observed+anas_tip_tip_observed,number_anas_theory

(array([12,  5, 10, 14, 16, 17, 11, 19, 14, 10, 10,  9, 16, 16,  8, 12,  9,
        14, 15,  7,  8]),
 [24,
  34,
  -14,
  25,
  36,
  20,
  33,
  3,
  33,
  37,
  6,
  22,
  15,
  25,
  15,
  38,
  32,
  36,
  0,
  5,
  14])

In [95]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(anas_tip_hyph_observed+anas_tip_tip_observed)
ax.plot(number_anas_theory)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …